In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import random
import joblib
import os
os.chdir('Resources/')

In [2]:
df = pd.read_csv('7_CC_Encrypted_Data.csv')
df = df.dropna()

In [3]:
Y = df[['HeartDisease']]
Y

,HeartDisease
1,1
3,1
8,1
11,1
13,1
...,...
2640,1
2641,0
2644,1
2658,1


In [4]:
c1_list = []
c2_list = []

for k in range(11):
    cnt = 0
    c1_sum = 0
    c2_sum = 0
    
    messages = df.iloc[:, k].values

    for i, message in enumerate(messages, 1):
        if message is not None and message == message:
            c1, c2 = map(int, message.strip("()").split(","))
            c1_sum += c1
            c2_sum += c2
            cnt += 1
    
    print(c1_sum//cnt, ",", c2_sum//cnt)

    c1_list.append(c1_sum//cnt)
    c2_list.append(c2_sum//cnt)

print(c1_list)
print(c2_list)

7725238297445332127407991595158267606450716631553 , 16769086783313564315005892769905829054995190029829
35517718033104704088502485275726341561551904284337 , 31694636052743011877949568441539763601955516573450
22443659489472860229040290952583046495774185357615 , 11407956384593602362165329678909474016590433695050
32645786427389652046014896610704699713743820311933 , 27848688656612519512115896382947924708080835729495
1304267054805981944467088018208132613735969340595 , 40213139556224060457987572335146069079275789945642
18959108316266119489233644530980633950698663352542 , 28082894499471325471937036264896652341526293338076
12741437569542050747073334897738270095511225090014 , 19660254220672399784068893752515660820093302012187
7384466023307569122962762536709314464704923088871 , 25542657224091022127356801313755505015993021832836
20461355034696913938060735031266347179123151849021 , 35928566874823627247307907597475510870209061128324
13381293829980648505678154992049712828324147825414 , 22796781457787

In [5]:
column_names = [
    "Age", "Sex", "ChestPainType", "RestingBloodPressure", "Cholesterol",
    "FastingBloodSugar", "RestingECG", "MaximumHeartRate", "ExerciseAngina",
    "Oldpeak", "ST_Slope", "HeartDisease"
]

In [6]:
imputed_data = pd.DataFrame()

for k in range(11):
    
    messages = df.iloc[:, k].values

    imputed_column = []

    for i, message in enumerate(messages, 1):
        if message is None:
            c1 = c1_list[k]
            c2 = c2_list[k]
        
        if message is not None and message == message:
            c1, c2 = map(int, message.strip("()").split(","))

        imputed_column.append(f"({c1},{c2})")

    imputed_data[column_names[k]] = imputed_column

In [7]:
scaler = joblib.load('13_CC_Scaler.joblib')

In [8]:
df_scaled = scaler.transform(imputed_data)
df_scaled = pd.DataFrame(df_scaled, columns=imputed_data.columns)
df_scaled.to_csv('14_CC_Scaled_Data.csv', index=False)

g:\Thesis - Paper\Resources\custom_tuple_scaler.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  c2_values = X.applymap(self._extract_c2)


In [9]:
file_path = "13_CC_F1_Score_RF_XGB.txt"

with open(file_path, "r") as f:
    reference_f1_score = float(f.read().strip())

In [10]:
import ast

with open('12_SP_Test_Data_RF_XGB.txt', 'r') as f:
    content = f.read().strip()
    test_indices = ast.literal_eval(content)[0]

X_test = df_scaled.iloc[test_indices]
Y_test = Y.iloc[test_indices]

In [16]:
import numpy as np
import joblib

model_dict = joblib.load('12_SP_Model_RF_XGB.joblib')
rf = model_dict['rf']
xgb = model_dict['xgb']

rf_pred = rf.predict(X_test)
xgb_pred = xgb.predict(X_test)

combined_pred = [r if r == x else r for r, x in zip(rf_pred, xgb_pred)]

combined_f1 = f1_score(Y_test, combined_pred) * 100
print(f"Combined F1-score: {combined_f1 : .2f}%")
print(f"Reference F1-score: {reference_f1_score * 100 : .2f}%")

Combined F1-score:  94.78%
Reference F1-score:  94.78%
